<a href="https://colab.research.google.com/github/bethelhall/cassava-disease-classifier/blob/master/Cassava_Disease_Classifer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
 import tensorflow as tf
import os
import numpy as np
import cv2  
import matplotlib.pyplot as plt

In [18]:
!git add 

fatal: pathspec 'drive/My' did not match any files


In [4]:
#mount Cassava Disease Dataset from Kaggle into your drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
print(len(os.listdir('/content/drive/My Drive/cassava-disease/train/')))

5


In [0]:
#Base directory of the training examples
TRAINING_BASE_DIR = '/content/drive/My Drive/cassava-disease/train/'

In [0]:
#Generate batches of tensor image data with real-time data augmentation.
#10% of the training data is used for validation data
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                                                shear_range = 0.1,
                                                                zoom_range = 0.1,
                                                                validation_split=0.1 
                                                                )

In [9]:
#Loop over in batches both for training and validation data
training_images = train_datagen.flow_from_directory(
    TRAINING_BASE_DIR,
    target_size = (150,150),
    shuffle=True,
    batch_size = 32,
    class_mode = 'categorical',
    subset="training"
)
testing_images = train_datagen.flow_from_directory(
    TRAINING_BASE_DIR,
    target_size = (150,150),
    batch_size = 32,
    class_mode = 'categorical',
    subset="validation"
)

Found 5351 images belonging to 5 classes.
Found 592 images belonging to 5 classes.


In [0]:
#import InceptionV3 and models and layers for an optional transer transfer learning
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import Model
from tensorflow.keras import layers

In [12]:
#Get inception weights
inception_weights = "/content/drive/My Drive/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
#Define InceptionV3
pre_trained_model = InceptionV3(include_top = False, input_shape =(150,150,3), weights = None)
#Load the weigths
pre_trained_model.load_weights(inception_weights)

#Freez all layers of the model
for layer in pre_trained_model.layers:
  layer.trainable = False
  
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [0]:
#Get the last ConV layer
last_layer = pre_trained_model.get_layer('mixed9')
print("Last layer output shape: ", last_layer.output_shape)
last_layer = last_layer.output
#Define the dense layers 
x = layers.Flatten()(last_layer)
x = layers.Dense(1024, activation = 'relu') (x)
x = layers.Dropout(.2)(x)
x = layers.Dense(5, activation = tf.nn.softmax)(x)

Last layer output shape:  (None, 3, 3, 2048)


In [0]:
#Design the model
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Conv2D(128, (3,3), activation = 'relu', input_shape=(150,150,3)),
                                    tf.keras.layers.MaxPool2D((2,2)),
                                    tf.keras.layers.Conv2D(64, (3,3),activation = 'relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(64,(3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512,activation = 'relu'),
                                    tf.keras.layers.Dense(5, activation=tf.nn.softmax)


])

model.compile(loss = 'categorical_crossentropy', optimizer= tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9), metrics=['accuracy'])

#Train the model with 50 epochs (The accuracy should be greater than 98.4%)
history = model.fit(
    training_images,
    epochs = 50, 
    verbose = 1,
    steps_per_epoch = 200,
    validation_data = testing_images,
    validation_steps = 100
    )


In [0]:
model.save('model.h5')

In [0]:
#Plot the loss VS accuracy graph to visualize how well the model performed in each epoch
loss_train = history.history['loss']
loss_val = history.history['val_accuracy']
epochs = range(len(accuracy))
plt.plot(epochs, loss_train, 'g', label='Training accuracy')
plt.plot(epochs, loss_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')  
plt.xlabel('Epochs')    j 
plt.ylabel('Accuracy')
plt.legend()
plt.show()